In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import FactorAnalyzer

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb
from lightgbm import LGBMClassifier 

from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import TruncatedSVD

import plotly.express as px
import plotly.graph_objs as go
import statsmodels.formula.api as smf
from plotly.graph_objects import Layout

In [2]:
df = pd.read_csv("mergedData.csv")

In [142]:
df = df.drop("Unnamed: 0", axis = 1)

In [143]:
df = df.drop(["DriverNumber", "Driver", "Points"], axis = 1)
# df = df.drop(["DriverNumber", "Driver", "Position"], axis = 1)
aux = df

In [144]:
# create new columns based on Status values
df['carIssue'] = (df['Status'] == 'carIssue').astype(int)
df['driverIssue'] = (df['Status'] == 'driverIssue').astype(int)
df = df.drop('Status', axis=1)

In [145]:
df["RaceCountry"] = df["RaceCountry"].str.replace(" ", "")
df["TeamName"] = df["TeamName"].str.replace(" ", "")
# df = df.drop(["Engine"], axis = 1)
df["Engine"] = df["Engine"].str.replace(" ", "")

In [146]:

# get a list of all categorical variables except "Abbreviation" and "raceID"
cat_vars = df.select_dtypes(include=['object']).columns.tolist()
cat_vars.remove("Abbreviation")
cat_vars.remove("RaceCountry")

# create dummy variables for all categorical variables except "Abbreviation" and "raceID"
dummies = pd.get_dummies(df[cat_vars], drop_first=True)

# drop the original categorical variables from the DataFrame
df = df.drop(cat_vars, axis=1)

# concatenate the dummy variables with the remaining variables in the DataFrame
df = pd.concat([df, dummies], axis=1)


In [148]:
#df = df.drop(["Year", "AverageRPM"], axis = 1)
df = df.drop(["AverageRPM"], axis = 1)
df = df.drop(["AvgLapTime"], axis = 1)
df = df.drop(["AvgSplitTime"], axis = 1)

In [149]:
df

,Abbreviation,Position,GridPosition,RaceCountry,Year,AgeAtGP,BestQualiTime,FLap,SDLapTime,AvgPitTime,...,TeamName_RacingPoint,TeamName_RedBullRacing,TeamName_Renault,TeamName_ToroRosso,TeamName_Williams,Engine_Honda,Engine_Mercedes,Engine_RedBull,Engine_Renault,CircuitType_street
0,GAS,11.0,17.0,Australia,2019.0,22.0,83.020,0.0,2.723962,21.2690,...,0,1,0,0,0,1,0,0,0,1
1,PER,13.0,10.0,Australia,2019.0,29.0,82.532,0.0,2.908751,23.2340,...,1,0,0,0,0,0,1,0,0,1
2,LEC,5.0,5.0,Australia,2019.0,21.0,81.442,0.0,2.795510,22.3060,...,0,0,0,0,0,0,0,0,0,1
3,STR,9.0,16.0,Australia,2019.0,20.0,83.017,0.0,2.711268,22.4710,...,1,0,0,0,0,0,1,0,0,1
4,MAG,6.0,7.0,Australia,2019.0,26.0,82.099,0.0,2.795637,22.3880,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,VET,10.0,9.0,AbuDhabi,2022.0,35.0,84.961,0.0,3.057131,23.1860,...,0,0,0,0,0,0,1,0,0,0
1543,SAI,4.0,4.0,AbuDhabi,2022.0,28.0,84.242,0.0,3.799668,21.7525,...,0,0,0,0,0,0,0,0,0,0
1544,LAT,19.0,20.0,AbuDhabi,2022.0,27.0,86.054,0.0,6.817590,23.7985,...,0,0,0,0,1,0,1,0,0,0
1545,RUS,5.0,6.0,AbuDhabi,2022.0,23.0,84.511,0.0,4.489247,26.0870,...,0,0,0,0,0,0,1,0,0,0


In [150]:
#df.to_csv(r'df_points.csv', index=True, header=True)
#df.to_csv(r'df_position.csv', index=True, header=True)
#df.to_csv(r'plotdata_points.csv', index=True, header=True)
#df.to_csv(r'plotdata_position.csv', index=True, header=True)

## MERF

In [152]:
from merf import MERF

In [157]:
# Split the data into two datasets based on the Year column
df_before = df[df['Year'].isin([2019, 2020])]
df_after = df[df['Year'].isin([2021, 2022])]

## Before model

In [158]:
df_before["raceID"]

0       1
1       1
2       1
3       1
4       1
       ..
717    37
718    37
719    37
720    37
721    37
Name: raceID, Length: 722, dtype: int64

In [159]:
# Split the data into train and test sets based on the raceID column
train_df = df_before[(df_before['raceID'] >= 1) & (df_before['raceID'] <= 29)]
test_df = df_before[(df_before['raceID'] >= 30) & (df_before['raceID'] <= 37)]

In [160]:
X_train = train_df.drop('Position', axis=1)
Y_train = train_df['Position']
X_test = test_df.drop('Position', axis=1)
Y_test = test_df['Position']

In [161]:
X_train = X_train.drop('RaceCountry', axis=1)
X_train = X_train.drop('Year', axis=1)
X_test = X_test.drop('RaceCountry', axis=1)
X_test = X_test.drop('Year', axis=1)

In [162]:
x_train = X_train.drop(['Abbreviation', 'raceID'], axis = 1)
Z_train = X_train[['raceID']]
clusters_train = X_train['raceID']

In [163]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

mrf = MERF(fixed_effects_model=model, max_iterations=30)

mrf.fit(x_train, Z_train, clusters_train, Y_train)


INFO     [merf.py:307] Training GLL is 1596.4646623331869 at iteration 1.
INFO     [merf.py:307] Training GLL is 1551.7884174971261 at iteration 2.
INFO     [merf.py:307] Training GLL is 1542.105886004856 at iteration 3.
INFO     [merf.py:307] Training GLL is 1538.0425715316378 at iteration 4.
INFO     [merf.py:307] Training GLL is 1536.2229205798524 at iteration 5.
INFO     [merf.py:307] Training GLL is 1535.3740519146759 at iteration 6.
INFO     [merf.py:307] Training GLL is 1534.9766138964126 at iteration 7.
INFO     [merf.py:307] Training GLL is 1534.7997873223103 at iteration 8.
INFO     [merf.py:307] Training GLL is 1534.732889336695 at iteration 9.
INFO     [merf.py:307] Training GLL is 1534.7207718824654 at iteration 10.
INFO     [merf.py:307] Training GLL is 1534.7353817557064 at iteration 11.
INFO     [merf.py:307] Training GLL is 1534.7622258925526 at iteration 12.
INFO     [merf.py:307] Training GLL is 1534.7937351805408 at iteration 13.
INFO     [merf.py:307] Training GLL 

In [164]:
train_preds = mrf.predict(x_train, Z_train, clusters_train)

In [165]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_train, train_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_train, train_preds)
r2 = r2_score(Y_train, train_preds)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)


# Calculate the RMSPE
mse = mean_squared_error(Y_train, train_preds)
rmspe = np.sqrt(mse / np.mean(Y_train**2))

# Print the RMSPE
print("Test RMSPE:", rmspe)

Test MSE: 7.087361983712685
Test RMSE: 2.6622099811458684
Test MAE: 2.0889718378656745
Test R-squared: 0.7823832377183746
Test RMSPE: 0.22529975232393407


# Test dataset

In [166]:
x_test = X_test.drop(['Abbreviation', 'raceID'], axis = 1)
Z_test = X_test[['raceID']]
clusters_test = X_test['raceID']

In [167]:
x_test

,GridPosition,AgeAtGP,BestQualiTime,FLap,SDLapTime,AvgPitTime,PitstopNo,HARD,INTERMEDIATE,MEDIUM,...,TeamName_RacingPoint,TeamName_RedBullRacing,TeamName_Renault,TeamName_ToroRosso,TeamName_Williams,Engine_Honda,Engine_Mercedes,Engine_RedBull,Engine_Renault,CircuitType_street
568,9.0,24.0,93.000,0.0,4.414441,29.5895,2.0,24.0,0.0,11.0,...,0,0,0,0,0,0,0,0,0,1
569,4.0,30.0,92.317,0.0,3.598099,29.6020,1.0,33.0,0.0,0.0,...,1,0,0,0,0,0,1,0,0,1
570,10.0,22.0,93.239,0.0,3.308658,30.2610,1.0,25.0,0.0,28.0,...,0,0,0,0,0,0,0,0,0,1
571,18.0,28.0,94.681,0.0,3.643243,30.1880,1.0,33.0,0.0,19.0,...,0,0,0,0,0,0,0,0,0,1
572,15.0,24.0,93.008,0.0,3.737778,30.1610,2.0,26.0,0.0,26.0,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,18.0,25.0,98.443,0.0,7.947119,21.8705,2.0,24.0,0.0,30.0,...,0,0,0,0,1,0,1,0,0,0
718,16.0,21.0,98.045,0.0,8.040811,22.0850,1.0,44.0,0.0,10.0,...,0,0,0,0,1,0,1,0,0,0
719,15.0,41.0,97.555,0.0,8.160983,22.0120,1.0,44.0,0.0,10.0,...,0,0,0,0,0,0,0,0,0,0
720,2.0,31.0,95.271,0.0,3.679513,21.5870,1.0,45.0,0.0,10.0,...,0,0,0,0,0,0,1,0,0,0


In [168]:
y_pred = mrf.predict(x_test, Z_test, clusters_test)

In [169]:
from sklearn.metrics import r2_score
mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)



Test MSE: 34.24464770089943
Test RMSE: 5.851892659721249
Test MAE: 3.869540405529234
Test R-squared: -0.09558628938852576


In [170]:
actual_values = Y_test
predictions = y_pred
# Calculate the mean of actual values
mean_actual = sum(actual_values) / len(actual_values)

# Calculate SSE and SST
sse = sum((actual - pred) ** 2 for actual, pred in zip(actual_values, predictions))
sst = sum((actual - mean_actual) ** 2 for actual in actual_values)

# Calculate R-squared
r2 = 1 - (sse / sst)
r2

-0.09558628938852576

In [171]:
# Calculate R-squared
r = np.corrcoef(predictions, actual_values)[0, 1]
r2 = r**2
r2

0.3716447876071426

In [126]:
Y_test

568     9.0
569     4.0
570     6.0
571    12.0
572    10.0
       ... 
717    17.0
718    15.0
719    12.0
720     2.0
721    16.0
Name: Position, Length: 154, dtype: float64

In [125]:
y_pred

array([  9.04458751,   7.76957566,   6.88491012,  14.85751325,
        10.02708778,   8.92706864,   8.38625701,   8.68988687,
         4.17879123,   9.98877399,   3.42938408,   8.68550655,
        16.4578558 ,  15.80292338,  13.97095574,   2.86743789,
        15.08901619,  13.20922514,   7.90857343,   8.3736811 ,
         4.26214832,  12.72749219,  14.09079764,   8.73684935,
        12.47617816,   6.69937475,  16.0988852 ,   2.91093933,
        14.62329593,   2.43472747,   7.66453815,   8.2361505 ,
        13.88298096,  23.80956035,  13.69971291,  11.39181322,
        12.52644823,  11.1431633 ,   6.55138959,   7.65614187,
         4.46145151,  18.33240336,  13.72076547,   5.27009525,
         9.87734549,   7.88737669,   8.87275754,   3.0871852 ,
         8.45110965,   1.08080962,   8.60005794,   6.75996915,
        15.67931405,  13.16422545,  11.53315005,   2.32945253,
        14.16212568,  12.77926804,  16.21960138,   9.843263  ,
         5.56210151,  12.7669672 ,  21.10131682,   6.62

In [57]:
train_preds

array([ 7.09094299,  9.6626787 ,  4.37229868, 10.6785708 , 10.77106915,
        9.85391016, 11.45216322,  9.4955561 , 19.42926404,  4.37506559,
        8.40914779,  1.22229485,  3.3357225 , 20.50552923, 15.00136858,
       10.35981784, -0.10011336, 19.70222717, 16.68558932, 10.55588871,
        7.48726851, 10.71922867,  3.04178925, 11.63765887, 11.32148316,
       11.10336371, 11.24216155, 18.78122784, 15.96490264,  4.90428735,
        8.61166053,  3.31583122,  5.84247723, 16.68779508, 15.35169091,
       11.14136782,  3.08817927, 21.55848094, 15.43459436, 12.66669648,
        5.67327942,  9.40950325,  5.63057141, 11.28919616, 13.54845154,
       19.70000068, 19.18218839,  7.51556153,  5.56336862, 17.79706994,
        2.7050926 ,  5.7249139 , 10.38712305, 15.53174972, 12.18411788,
        2.66864098, 13.26205182, 14.88504629, 12.32986107, 12.16537093,
        7.13555283,  5.59692437,  9.43057629, 13.56227735, 10.41932955,
       17.12633186, 11.15625358, 17.97234522,  3.9664317 ,  8.57

In [24]:
df.columns

Index(['Abbreviation', 'Position', 'GridPosition', 'RaceCountry', 'Year',
       'AgeAtGP', 'BestQualiTime', 'FLap', 'SDLapTime', 'AvgPitTime',
       'PitstopNo', 'HARD', 'INTERMEDIATE', 'MEDIUM', 'SOFT', 'WET', 'Rain',
       'AverageSpeed', 'MaxSpeed', 'MaxRPM', 'AverageThrottle',
       'MaxThrottlePct', 'Brake', 'raceID', 'carIssue', 'driverIssue',
       'after_2020', 'TeamName_AlfaRomeoRacing', 'TeamName_AlphaTauri',
       'TeamName_Alpine', 'TeamName_AstonMartin', 'TeamName_Ferrari',
       'TeamName_HaasF1Team', 'TeamName_McLaren', 'TeamName_Mercedes',
       'TeamName_RacingPoint', 'TeamName_RedBullRacing', 'TeamName_Renault',
       'TeamName_ToroRosso', 'TeamName_Williams', 'CircuitType_street'],
      dtype='object')

## After model

In [172]:
df_after["raceID"]

722     38
723     38
724     38
725     38
726     38
        ..
1542    80
1543    80
1544    80
1545    80
1546    80
Name: raceID, Length: 825, dtype: int64

In [173]:
df_after

,Abbreviation,Position,GridPosition,RaceCountry,Year,AgeAtGP,BestQualiTime,FLap,SDLapTime,AvgPitTime,...,TeamName_RacingPoint,TeamName_RedBullRacing,TeamName_Renault,TeamName_ToroRosso,TeamName_Williams,Engine_Honda,Engine_Mercedes,Engine_RedBull,Engine_Renault,CircuitType_street
722,GAS,17.0,5.0,Bahrain,2021.0,24.0,89.809,0.0,9.870618,29.212667,...,0,0,0,0,0,0,0,0,0,0
723,PER,5.0,20.0,Bahrain,2021.0,31.0,90.659,0.0,9.238201,24.096333,...,0,1,0,0,0,1,0,0,0,0
724,ALO,19.0,9.0,Bahrain,2021.0,39.0,90.249,0.0,9.050427,24.574000,...,0,0,0,0,0,0,0,0,1,0
725,LEC,6.0,4.0,Bahrain,2021.0,23.0,89.678,0.0,7.326363,24.550500,...,0,0,0,0,0,0,0,0,0,0
726,STR,10.0,10.0,Bahrain,2021.0,22.0,90.601,0.0,9.904268,25.204500,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,VET,10.0,9.0,AbuDhabi,2022.0,35.0,84.961,0.0,3.057131,23.186000,...,0,0,0,0,0,0,1,0,0,0
1543,SAI,4.0,4.0,AbuDhabi,2022.0,28.0,84.242,0.0,3.799668,21.752500,...,0,0,0,0,0,0,0,0,0,0
1544,LAT,19.0,20.0,AbuDhabi,2022.0,27.0,86.054,0.0,6.817590,23.798500,...,0,0,0,0,1,0,1,0,0,0
1545,RUS,5.0,6.0,AbuDhabi,2022.0,23.0,84.511,0.0,4.489247,26.087000,...,0,0,0,0,0,0,1,0,0,0


In [174]:
# Split the data into train and test sets based on the raceID column
train_df = df_after[(df_after['raceID'] >= 38) & (df_after['raceID'] <= 72)]
test_df = df_after[(df_after['raceID'] >= 73) & (df_after['raceID'] <= 80)]

In [175]:
X_train = train_df.drop('Position', axis=1)
Y_train = train_df['Position']
X_test = test_df.drop('Position', axis=1)
Y_test = test_df['Position']

In [176]:
X_train = X_train.drop('RaceCountry', axis=1)
X_train = X_train.drop('Year', axis=1)
X_test = X_test.drop('RaceCountry', axis=1)
X_test = X_test.drop('Year', axis=1)

In [177]:
x_train = X_train.drop(['Abbreviation', 'raceID'], axis = 1)
Z_train = X_train[['raceID']]
clusters_train = X_train['raceID']

In [178]:
#mrf = MERF(fixed_effects_model = LinearRegression(), max_iterations=30)
mrf.fit(x_train, Z_train, clusters_train, Y_train)

INFO     [merf.py:307] Training GLL is 1798.3726199915113 at iteration 1.
INFO     [merf.py:307] Training GLL is 1774.986836037651 at iteration 2.
INFO     [merf.py:307] Training GLL is 1770.384372461241 at iteration 3.
INFO     [merf.py:307] Training GLL is 1766.8062783736425 at iteration 4.
INFO     [merf.py:307] Training GLL is 1763.7753608996136 at iteration 5.
INFO     [merf.py:307] Training GLL is 1761.1336138538284 at iteration 6.
INFO     [merf.py:307] Training GLL is 1758.7881329778447 at iteration 7.
INFO     [merf.py:307] Training GLL is 1756.6771808268707 at iteration 8.
INFO     [merf.py:307] Training GLL is 1754.7570908973603 at iteration 9.
INFO     [merf.py:307] Training GLL is 1752.9956196284452 at iteration 10.
INFO     [merf.py:307] Training GLL is 1751.3681702713134 at iteration 11.
INFO     [merf.py:307] Training GLL is 1749.8555013718844 at iteration 12.
INFO     [merf.py:307] Training GLL is 1748.4422572239962 at iteration 13.
INFO     [merf.py:307] Training GLL 

In [179]:
train_preds = mrf.predict(x_train, Z_train, clusters_train)

In [180]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_train, train_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_train, train_preds)
r2 = r2_score(Y_train, train_preds)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)

Test MSE: 8.27348915822779
Test RMSE: 2.8763673545338033
Test MAE: 2.259793609210368
Test R-squared: 0.7382129081749542


# Test Dataset

In [181]:
x_test = X_test.drop(['Abbreviation', 'raceID'], axis = 1)
Z_test = X_test[['raceID']]
clusters_test = X_test['raceID']

In [182]:
y_pred = mrf.predict(x_test, Z_test, clusters_test)

In [183]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)


Test MSE: 7.302172787094027
Test RMSE: 2.7022532795972376
Test MAE: 2.1726833375904095
Test R-squared: 0.7686064777239628


In [184]:
actual_values = Y_test
predictions = y_pred
# Calculate the mean of actual values
mean_actual = sum(actual_values) / len(actual_values)

# Calculate SSE and SST
sse = sum((actual - pred) ** 2 for actual, pred in zip(actual_values, predictions))
sst = sum((actual - mean_actual) ** 2 for actual in actual_values)

# Calculate R-squared
r2 = 1 - (sse / sst)
r2

0.768606477723963